In [17]:
!pip install langchain

In [18]:
from langchain import HuggingFaceHub

HUGGINGFACEHUB_API_TOKEN = ""
llm = HuggingFaceHub(huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN, repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [19]:
question = "What is a Large Language Model?"
print(llm.predict(question))

a large language model is a model of language that is used to describe the structure and function of a language.


###Prompt Template And LLMChain

In [20]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'],
template = "Tell me the capital of {country}")

prompt_template.format(country="German")

'Tell me the capital of German'

In [21]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template)
chain.run("German")

'berlin'

###Combining Multiple Chains using Simple Sequential Chain

In [22]:
capital_template = PromptTemplate(input_variables=["country"],
template = "Tell me the capital of {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_template)

famous_template = PromptTemplate(input_variables=["capital"],
template = "Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [23]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("German")

'Berlin Zoo'

###Sequential Chain

In [24]:
capital_template = PromptTemplate(input_variables=["country"],
template = "Tell me the capital of {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")

famous_template = PromptTemplate(input_variables=["capital"],
template = "Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="place")

In [25]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains=[capital_chain, famous_chain],
                        input_variables=["country"],
                        output_variables=["capital", "place"])

chain({"country":"German"})

{'country': 'German', 'capital': 'berlin', 'place': 'Berlin Zoo'}

###Schema

In [26]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

llm([SystemMessage(content="You are a comedian AI assistant"),
     HumanMessage(content="Please provide some comedy punchlines about AI")])

ValueError: ignored

###Output Parser

In [27]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeperatedOutput(BaseOutputParser):
  def parse(self, text:str):
    return text.strip().split(",")

template = "You are a helpful assistant. When using any input, you must generate 5 synonym words in a comma separated list"
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)])

chain = chat_prompt|llm|CommaSeperatedOutput()

chain.invoke({"text":"intelligent"})

['intelligent human']